# Project 2

Griffith and Becca

# Q: How does not getting an appropriate of sleep effect the spread of sickness?

An inappropriate amount of sleep - either too little or too much - can weaken the immune system. This can make it easier for individuals to become infected by a sickness. This model aims to determine how having different percentages of the population with different amounts of sleep affects the spread of a sickness accross that population.

# To Do
* Fix make_system to work such that infected and susceptible add up to 1000, right now they add up to more.
* Survey with Olin students, how much sleep do they get?
* Implement number of Olin students into total enrollment (335). Make sure to prevent fractional students!

In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

This model relies on a lot of stocks (11 to be specific). There are susceptible and infected stocks for all sleep values (As the contact rate is different for each).

In [11]:
def make_system(sleepFive, sleepSix, sleepSeven, sleepEight, sleepNine, infectedInitial, beta5, beta6, beta7, beta8, beta9, beta, gamma):
    """Make a system object for the SIR model.
    
    sleepFive: Percentage of the population with five or less hours of sleep
    sleepSix: Percentage of the population with six hours of sleep
    sleepSeven: Percentage of the population with seven hours of sleep
    sleepEight: Percentage of the population with eight hours of sleep
    sleepNine: Percentage of the population with nine or more hours of sleep
    infectedInitial: Percentage of the population that starts out infected
    
    beta5: contact rate multiplier in for five hours of sleep
    beta6: contact rate multiplier for six hours of sleep
    beta7: contact rate multiplier for seven hours of sleep
    beta8: contact rate multiplier for eight hours of sleep
    beta9: contact rate multiplier for nine hours of sleep
    beta: contact rate in days
    gamma: recovery rate in days
    
    returns: System object
    """
    TotalPopulation = 1000
    SusceptiblePopulation = 1000 - (initialInfected * TotalPopulation)
    
    S5 = sleepFive * SusceptiblePopulation
    S6 = sleepSix * SusceptiblePopulation
    S7 = sleepSeven * SusceptiblePopulation
    S8 = sleepEight * SusceptiblePopulation
    S9 = sleepNine *  SusceptiblePopulation 
    
    I5 = sleepFive * TotalPopulation * infectedInitial
    I6 = sleepSix * TotalPopulation * infectedInitial
    I7 = sleepSeven * TotalPopulation * infectedInitial
    I8 = sleepEight * TotalPopulation * infectedInitial
    I9 = sleepNine * TotalPopulation * infectedInitial
    
    init = State(S5=S5, S6=S6, S7=S7, S8=S8, S9=S9, 
                 I5=I5, I6=I6, I7=I7, I8=I8, I9=I9, 
                 R=0)
    t0 = 0
    t_end = 100
    

    return System(init=init, t0=t0, t_end=t_end,
                  beta5=beta5, beta6=beta6, beta7=beta7, beta8=beta8, beta9=beta9,
                  beta=beta, gamma=gamma)

There are two plot functions. One plots the total Susceptible, Infected, and Recovered (SIR) and prints out a statement with the percentages of each. The other plot function plots all of the stocks (All of the various sleep values for susceptible and infected).

# FIX PLOT FUNCTION and total infected (If needed)
* Add a way to display the fraction that is each sleep group
* add recovered based on sleep

In [12]:
def plot_results_all(results):
    """Plot the results of a SIR model.
    
    results: Dataframe with the results of the model
    """
    TotalSusceptible = results.S5 + results.S6 + results.S7 + results.S8 + results.S9
    TotalInfected = results.I5 + results.I6 + results.I7 + results.I8 + results.I9
    
    plot(TotalSusceptible, '--', label='Total Susceptible')
    plot(TotalInfected, '-', label='Total Infected')
    plot(results.R, ':', label='Total Recovered')
    decorate(xlabel='Time (days)',
             ylabel='Population')

In [13]:
def plot_results_seperate(results):
    """PLot the results of a SIR model with seperate groups.
    
    results: Dataframe with the results of the model"""

In [14]:
def calc_total_infected(results):
    """Fraction of population infected during the simulation.
    
    results: DataFrame with columns S, I, R
    
    returns: fraction of population
    """
    return get_first_value(results.S) - get_last_value(results.S)

Uses the make_system function to make a system with the below fractions of people who get five, six, seven, eight, and nine hours of sleep. Also specify the portion of the population that starts out infected.

In [15]:
sleepFive = 0.052
sleepSix = 0.233
sleepSeven = 0.427
sleepNine = 0.05
sleepEight = 1 - sleepFive - sleepSix - sleepSeven - sleepNine
initialInfected = .05

#Numbers made up currently
beta5 = 1.7 #contact rate multiplier in for five hours of sleep
beta6 = 1.29 #contact rate multiplier for six hours of sleep
beta7 = 1.15 #contact rate multiplier for seven hours of sleep
beta8 = 1.0 #contact rate multiplier for eight hours of sleep
beta9 = 1.49 #contact rate multiplier for nine hours of sleep
beta = 0.333
gamma = 0.25
system = make_system(sleepFive, sleepSix, sleepSeven, sleepEight, sleepNine, initialInfected, beta5, beta6, beta7, beta8, beta9, beta, gamma)

,values
init,S5 49.40 S6 221.35 S7 405.65 S8 2...
t0,0
t_end,100
beta5,1.7
beta6,1.29
beta7,1.15
beta8,1
beta9,1.49
beta,0.333
gamma,0.25


Why does the susceptible require a ratio of susceptible people in that group to total susceptible people?

Why does infected not?

In [16]:
def slope_func(state, t, system):
    """Update the SIR model.
    
    state: State (s5, s6, s7, s8, s9, i5, i6, i7, i8, i9, r)
    t: time
    system: System object
    
    returns: pair of derivatives
    """ 
    s5, s6, s7, s8, s9, i5, i6, i7, i8, i9, r = state
    unpack(system)
    
    s = s5 + s6 + s7 + s8 + s9 # Total susceptible
    i = i5 + i6 + i7 + i8 + i9 # Total infected
    
    #VERIFY: Check: total change in infected = -(total change in susceptible) - (total change in recovered)
    ds5dt = -(beta*beta5) * i * (s5/s) #The change in the 5 hour of susceptible sleep group = infection rate * total infected * percent of susceptible that have 5 hours of sleep
    di5dt = -ds5dt - gamma * i5 #Change in 5 hour of sleep infected group = change in susceptible group becoming infected - (recovery rate * size of group with five hours of sleep)
    
    ds6dt = -(beta*beta6) * i * (s6/s)
    di6dt = -ds6dt - gamma * i6
    
    ds7dt = -(beta*beta7) * i * (s7/s)
    di7dt = -ds7dt - gamma * i7
    
    ds8dt = -(beta*beta8) * i * (s8/s)
    di8dt = -ds8dt - gamma * i8
    
    ds9dt = -(beta*beta9) * i * (s9/s)
    di9dt = -ds9dt - gamma * i9

    drdt = gamma * i
    
    return ds5dt, di5dt, ds6dt, di6dt, ds7dt, di7dt, ds8dt, di8dt, ds9dt, di9dt, drdt

In [17]:
slope_func(system.init, system.t0, system)

(-1.4718600000000002,
 0.8218600000000001,
 -5.004490500000001,
 2.091990500000001,
 -8.1759825,
 2.8384824999999996,
 -3.9627000000000003,
 0.9877000000000002,
 -1.240425,
 0.6154250000000001,
 12.5)

In [61]:
def update_func(state, dt, system):
    """Update the SIR model.
    
    state: State (s5, s6, s7, s8, s9, i5, i6, i7, i8, i9, r)
    t: time
    system: System object
    
    returns: pair of derivatives
    """ 
    s5, s6, s7, s8, s9, i5, i6, i7, i8, i9, r = state
    unpack(system)
    
    s = s5 + s6 + s7 + s8 + s9 # Total susceptible
    i = i5 + i6 + i7 + i8 + i9 # Total infected
    
    
    #VERIFY: Check: total change in infected = -(total change in susceptible) - (total change in recovered)
    s5 += (-(beta*beta5) * i * (s5/s)) * dt #The change in the 5 hour of susceptible sleep group = infection rate * total infected * percent of susceptible that have 5 hours of sleep
    i5 += ((beta*beta5) * i * (s5/s) - gamma * i5) * dt #Change in 5 hour of sleep infected group = change in susceptible group becoming infected - (recovery rate * size of group with five hours of sleep)
    
    s6 += (-(beta*beta6) * i * (s6/s)) * dt
    i6 += ((beta*beta6) * i * (s6/s) - gamma * i6) * dt
    
    s7 += (-(beta*beta7) * i * (s7/s)) * dt
    i7 += ((beta*beta7) * i * (s7/s) - gamma * i7) * dt
    
    s8 += (-(beta*beta8) * i * (s8/s)) * dt
    i8 += ((beta*beta8) * i * (s8/s) - gamma * i8) * dt
    
    s9 += (-(beta*beta9) * i * (s9/s)) * dt
    i9 += ((beta*beta9) * i * (s9/s) - gamma * i9) * dt

    r += (gamma * i) * dt
    
    return State(S5 = s5, I5=i5, S6=s6, I6=i6, S7=s7, I7=i7, S8=s8, I8=i8, S9=s9, I9=i9, R=r)

In [62]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[t0] = init
    
    dt = 1
    
    for t in linrange(t0, t_end, dt):
        frame.row[t+dt] = update_func(frame.row[t], dt, system)
    
    return frame

In [63]:
results = run_simulation(system, update_func)

,S5,S6,S7,S8,S9,I5,I6,I7,I8,I9,R
0,4.940000e+01,2.213500e+02,4.056500e+02,226.100000,47.500000,2.600000,11.650000,21.350000,11.900000,2.500000,0.000000e+00
1,4.792814e+01,2.163455e+02,3.974740e+02,222.137300,46.259575,3.378006,13.628844,24.023693,12.818248,3.083032,1.250000e+01
2,4.626745e+01,2.106572e+02,3.881575e+02,217.609676,44.854703,4.136652,15.760422,27.115953,14.049028,3.674481,2.673296e+01
3,4.439914e+01,2.042022e+02,3.775544e+02,212.440717,43.267183,4.895357,18.077445,30.650374,15.582949,4.287194,4.291709e+01
4,4.230448e+01,1.968918e+02,3.655049e+02,206.545112,41.478080,5.667358,20.606768,34.652699,17.419204,4.930520,6.129042e+01
5,3.996567e+01,1.886319e+02,3.518355e+02,199.828146,39.468232,6.460020,23.368477,39.147703,19.562929,5.610349,8.210956e+01
6,3.736714e+01,1.793252e+02,3.363606e+02,192.185421,37.219045,7.274591,26.373899,44.155081,22.022615,6.328773,1.056469e+02
7,3.449727e+01,1.688743e+02,3.188852e+02,183.502932,34.713655,8.105404,29.622281,49.683775,24.807195,7.083320,1.321857e+02
8,3.135086e+01,1.571864e+02,2.992102e+02,173.657712,31.938617,8.938491,33.095664,55.723894,27.922405,7.865690,1.620112e+02
9,2.793244e+01,1.441808e+02,2.771403e+02,162.519357,28.886302,9.749552,36.751296,62.234970,31.365747,8.659878,1.953977e+02


In [ ]:
results, details = run_ode_solver(system, slope_func)
details

In [ ]:
results.plot()

In [6]:
results, details = run_ode_solver(system, slope_func, max_step = 2)
details

NameError: name 'system' is not defined

In [51]:
results.plot()

NameError: name 'results' is not defined